In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type
import sqlite3
from sqlalchemy import create_engine

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts      

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [5]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    # gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=5)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [19]:
# 27 EV sessions per weekday
events_27_ev = get_synth_events(27)

In [20]:
# Checking session details
session_copy = deepcopy(events_27_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

651 session_21 651 723 25.04976199002258 Plugin
652 session_4 652 767 14.337668204305237 Plugin
660 session_11 660 787 7.448861832744039 Plugin
659 session_1 659 766 49.318023951792 Plugin
654 session_7 654 791 11.744675733652233 Plugin
664 session_12 664 783 3.612872074158752 Plugin
662 session_14 662 760 6.546060822680286 Plugin
660 session_15 660 769 6.397431305694834 Plugin
664 session_8 664 751 6.052569238809784 Plugin
661 session_0 661 729 0.7220315493831958 Plugin
654 session_20 654 782 10.534004955172477 Plugin
664 session_24 664 778 15.621889848371865 Plugin
666 session_5 666 789 8.727695259860116 Plugin
726 session_6 726 790 2.868310750898407 Plugin
663 session_13 663 779 7.392312377314358 Plugin
674 session_3 674 734 8.628516825648854 Plugin
666 session_16 666 786 19.33316674947693 Plugin
762 session_17 762 822 6.174831226212631 Plugin
777 session_18 777 837 4.408174749430781 Plugin
767 session_19 767 827 8.541232472931483 Plugin
705 session_9 705 765 21.93995016236091 Plugi

#### Algorithms and new equation

In [8]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

#### Run Simulation with New Equation

In [21]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2025, 4, 2))
    end = timezone.localize(datetime(2025, 4, 2))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_27_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_4", "session_12", "session_2"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [22]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            try:
                sims[config] = run_experiment(*config)
            except Exception as e:
                print(f"InfeasibilityException: {e}")
                sims[config] = None
                continue

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.2998528480529785
('RR', 80)
64.2570281124498
115.94202898550725
InfeasibilityException: 'RR'
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 11.41680645942688
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.30001163482666016
('RR', 85)
68.27309236947791
123.18840579710144
InfeasibilityException: 'RR'
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 11.483523607254028
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.300243616104126
('RR', 90)
72.28915662650603
130.43478260869566
InfeasibilityException: 'RR'
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 11.292280197143555
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.3000485897064209
('RR', 95)
76.30522088353413
137.68115942028987
InfeasibilityException: 'RR'
('MPC', 95)
76.30522088353413
137.68115942028987
Running...
R

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [24]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Energy Delivered to priority evs (%)": analysis.proportion_of_priority_evs_energy_delivered(sim, ["session_4",
                                                                                                           "session_7",
                                                                                                           "session_15",
                                                                                                           "session_1",
                                                                                                           "session_19",
                                                                                                           "session_20",
                                                                                                           "session_22",
                                                                                                           ]),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [29]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [30]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [31]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,123.750,27.947,376.593,100.0,99.00,0.377
1,three_phase,MPC,80,85.064,62.963,82.500,24.987,320.344,100.0,66.00,0.384
2,three_phase,Unctrl,85,100.000,100.000,116.471,27.947,376.593,100.0,99.00,0.355
3,three_phase,MPC,85,84.803,55.556,77.647,24.953,319.362,100.0,66.00,0.383
4,three_phase,Unctrl,90,100.000,100.000,110.000,27.947,376.593,100.0,99.00,0.350
5,three_phase,MPC,90,84.941,59.259,77.022,24.919,319.882,100.0,69.32,0.385
6,three_phase,Unctrl,95,100.000,100.000,104.211,27.947,376.593,100.0,99.00,0.364
7,three_phase,MPC,95,84.812,59.259,72.968,24.877,319.398,100.0,69.32,0.391
8,three_phase,Unctrl,100,100.000,100.000,99.000,27.947,376.593,100.0,99.00,0.356
9,three_phase,MPC,100,84.757,59.259,69.320,24.859,319.190,100.0,69.32,0.392


#### Simulation with different algorithms

In [32]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev, 2),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [33]:
sims = dict()

In [34]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.3132758140563965
('LLF', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.7697420120239258
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 11.197873592376709
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.30889415740966797
('LLF', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.7822246551513672
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 11.92759084701538
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.3329644203186035
('LLF', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.8002111911773682
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 11.516714334487915
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.2834794521331787
('LLF', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.7441363334655762
('MPC', 95)


In [35]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [36]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,123.750,27.947,376.593,100.000,99.000,0.365
1,three_phase,LLF,80,80.118,81.481,87.944,22.731,301.719,99.867,70.355,0.392
2,three_phase,MPC,80,84.569,55.556,82.500,24.876,318.481,100.000,66.000,0.379
3,three_phase,Unctrl,85,100.000,100.000,116.471,27.947,376.593,100.000,99.000,0.373
4,three_phase,LLF,85,84.508,81.481,87.165,24.088,318.252,99.883,74.090,0.396
5,three_phase,MPC,85,84.555,55.556,77.647,24.868,318.429,100.000,66.000,0.407
6,three_phase,Unctrl,90,100.000,100.000,110.000,27.947,376.593,100.000,99.000,0.356
7,three_phase,LLF,90,75.052,74.074,83.478,21.153,282.641,99.721,75.130,0.418
8,three_phase,MPC,90,85.138,66.667,77.022,24.983,320.624,100.000,69.320,0.408
9,three_phase,Unctrl,95,100.000,100.000,104.211,27.947,376.593,100.000,99.000,0.376
